# HW4 - Breast Cancer Detection using Multiple Instance Learning (MIL) (100 points)

## Course Name: Intelligent Analysis of Biomedical Images

#### Lecturers: Dr. Rohban
#### Name: Amir Mohammad Ezzati
#### Student ID: 402212269

---

**Contact**: Ask your questions in Quera

---




In this part we are going implement the idea proposed by the paper "Breast Cancer Histopathology Image Classification
and Localization using Multiple Instance Learning" and reproduce some of their results. Please read this paper before starting the next part. Here is the link to it:

https://arxiv.org/pdf/2003.00823.pdf

In the next sections, we only use one of the presented datasets in the above paper. In the explanation of every section, it is assumed that you have read the paper before starting that part. It should also be mentioned that you are free to modify everything in this notebook and all provided codes are for better understanding.

You can read the "Attention-based Deep Multiple Instance Learning" paper for more explanation. Here is the link to it:

https://arxiv.org/pdf/1802.04712.pdf

## Imports

Feel free to import any library you need.

In [1]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.0 MB/s eta 0:00:00


In [2]:
import torch
from torch import nn, optim
import numpy as np
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import os
from PIL import Image
from torch.utils.data import random_split
from torchvision import transforms as T
from torch.autograd import Variable
import torch.nn.functional as F
import wandb
import IPython

In [3]:
wandb.login(key='f63f6bf237335da137544dff247f145d2db7f47a')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## Downloading dataset

Here is the code to download and decompress the BreaKHis dataset. This code is written for the google colab environment and you may need to modify it for running on other devices.

Link to the dataset's website:

https://web.inf.ufpr.br/vri/databases/breast-cancer-histopathological-database-breakhis/

In [62]:
!wget http://www.inf.ufpr.br/vri/databases/BreaKHis_v1.tar.gz -O /content/BreaKHis_v1.tar.gz

--2023-12-26 14:08:32--  http://www.inf.ufpr.br/vri/databases/BreaKHis_v1.tar.gz
Resolving www.inf.ufpr.br (www.inf.ufpr.br)... 200.17.202.113, 2801:82:80ff:8001:216:ccff:feaa:79
Connecting to www.inf.ufpr.br (www.inf.ufpr.br)|200.17.202.113|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.inf.ufpr.br/vri/databases/BreaKHis_v1.tar.gz [following]
--2023-12-26 14:08:34--  https://www.inf.ufpr.br/vri/databases/BreaKHis_v1.tar.gz
Connecting to www.inf.ufpr.br (www.inf.ufpr.br)|200.17.202.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4273561758 (4.0G) [application/octet-stream]
Saving to: ‘/content/BreaKHis_v1.tar.gz’

/content/BreaKHis_v 100%[===================>]   3.98G  9.19MB/s    in 7m 38s  

2023-12-26 14:16:14 (8.90 MB/s) - ‘/content/BreaKHis_v1.tar.gz’ saved [4273561758/4273561758]



In [63]:
!tar xzf /content/BreaKHis_v1.tar.gz -C /content

## Data Preparation (35 points)

In this section, you will implement several things:

- A custom pytorch Dataset for our data. You can use the current decompressed folder or change the structure to make it easier to work with.
- Transformations to perform on data (You must first do the patch extraction, then apply the transformations on every patch.)
- Patch extraction code (you can also implement it in the following sections)
- Dataloaders (instead of 80-20 split for train-test sets from the paper, you should use 80-5-15 split for train-val-test sets here and save best model w.r.t. the validation loss)

In [75]:
data_path = '/content/BreaKHis_v1/histology_slides/breast'

classes = ['benign', 'malignant']
magnifications = ["40X", "100X", "200X", "400X"]

data = {
    x: [] for x in magnifications
}

for _class in classes:
    class_path = os.path.join(data_path, _class, 'SOB')

    for status in os.listdir(class_path):
        status_path = os.path.join(class_path, status)

        for sample in os.listdir(status_path):
            sample_path = os.path.join(status_path, sample)

            for magnification in os.listdir(sample_path):
                magnification_path = os.path.join(sample_path, magnification)

                for patch in os.listdir(magnification_path):
                    image_path = os.path.join(magnification_path, patch)
                    if _class == 'benign':
                      data[magnification].append([image_path, 0])
                    else: # malignant
                      data[magnification].append([image_path, 1])

In [76]:
print(data.keys())

dict_keys(['40X', '100X', '200X', '400X'])


In [77]:
# Implement the custom dataset here (15 points)
class CustomBreaKHis(Dataset):
    def __init__(self, data, magn, transform=None):
      self.data = np.array(data)
      self.magnification = magn
      self.transform = transform

    def __len__(self):
      return self.data.shape[0]

    def __getitem__(self, index):
      image_path, label = self.data[index]

      image = Image.open(image_path)

      # patch extraction section
      crop = T.Compose([T.CenterCrop((448,700))])
      image = np.array(crop(image))

      patch_list = []
      patch_size = 28
      rows, cols, _ = image.shape
      for i in range(0, rows, patch_size):
          for j in range(0, cols, patch_size):
              patch_list.append(self.transform(image[i:i+patch_size, j:j+patch_size, :]))

      # Convert the list of patches to a tuple
      patch_tuple = tuple(patch_list)

      # Stack the patches using torch
      stacked_patches = torch.stack(patch_tuple, 0)

      label = torch.from_numpy(np.array(label, dtype=int))

      return stacked_patches, label

In [78]:
# Implement the transforms here (5 points)
# all transforms in paper
toPIL = T.Compose([
    T.ToPILImage()
])
crop = T.Compose([T.CenterCrop((448,700))])

transformers = [
    T.functional.vflip,
    T.functional.hflip,
    T.RandomRotation((90, 90)),
    T.RandomRotation((180, 180)),
    T.RandomRotation((270, 270))
]
augmentation = T.RandomApply([T.RandomChoice(transformers)], p=0.5)

transforms = T.Compose([
    T.ToTensor(),
])

In [79]:
# Create dataloaders here (5 points)
magnifications = ["40X", "100X", "200X", "400X"]

dataset = { x: {} for x in magnifications }

# torch.manual_seed(7)
for magn in magnifications:
  ds = CustomBreaKHis(data[magn], magn, transform=transforms)
  dataset[magn]['train'], dataset[magn]['valid'], dataset[magn]['test'] = random_split(ds, [0.8, 0.05, 0.15])


dataloader = { x: {} for x in magnifications }

for magn in magnifications:
  for phase in ['train', 'valid', 'test']:
    if phase == 'train':
      dataloader[magn][phase] =  DataLoader(dataset[magn][phase], shuffle=True, batch_size=1)
    else:
      dataloader[magn][phase] =  DataLoader(dataset[magn][phase], shuffle=False, batch_size=1)

In [80]:
# Implement patch extraction here (10 points)
# If you are going to implement it somewhere else, please comment it here.

# *** patch extraction is implemented in custom dataset

## Model Implementation (25 points)

Here, you have to implement the proposed architecture in the paper. The only point is that in the last part of the model, instead of a "dense+softmax" you should implement a "dense+sigmoid".

In [12]:
# Implement the model here (25 points)

class Attention(nn.Module):
    def __init__(self):
        super(Attention, self).__init__()
        self.L = 500
        self.D = 128
        self.K = 1

        self.feature_extractor_part1 = nn.Sequential(
            nn.Conv2d(3, 20, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(20, 50, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2)
        )

        self.feature_extractor_part2 = nn.Sequential(
            nn.Linear(50 * 4 * 4, self.L),
            nn.ReLU(),
        )

        self.attention = nn.Sequential(
            nn.Linear(self.L, self.D),
            nn.Tanh(),
            nn.Linear(self.D, self.K)
        )

        self.classifier = nn.Sequential(
            nn.Linear(self.L*self.K, 1),
            nn.Sigmoid()
        )

    def forward(self, x):

        x = x.squeeze(0)

        H = self.feature_extractor_part1(x)
        H = H.view(-1, 50 * 4 * 4)
        H = self.feature_extractor_part2(H)  # NxL

        A = self.attention(H)  # NxK
        A = torch.transpose(A, 1, 0)  # KxN
        A = F.softmax(A, dim=1)  # softmax over N

        M = torch.mm(A, H)  # KxL

        Y_prob = self.classifier(M)
        Y_hat = torch.ge(Y_prob, 0.5).float()

        return Y_prob, Y_hat, A

    # AUXILIARY METHODS
    def calculate_classification_error(self, X, Y):
        Y = Y.float()
        _, Y_hat, _ = self.forward(X)
        error = 1. - Y_hat.eq(Y).cpu().float().mean().item()

        return error, Y_hat

    def calculate_objective(self, X, Y):
        Y = Y.float()
        Y_prob, _, A = self.forward(X)
        Y_prob = torch.clamp(Y_prob, min=1e-5, max=1. - 1e-5)
        neg_log_likelihood = -1. * (Y * torch.log(Y_prob) + (1. - Y) * torch.log(1. - Y_prob))  # negative log bernoulli

        return neg_log_likelihood, A

## Training (32 points)

In this section, you will implement the training method and use it to train the model for every magnification. The model has to be trained for at least 60 epochs. But, in every epoch, we use 200 mini-batches to train the model instead of the whole training data, which means with a batch size of one, we use 200 images for training the model in every epoch. With this method, we spend less training time but actually, iterate whole data several times. (You have to make sure that the train loader will get shuffled correctly every epoch)

You should save loss and accuracy per epoch for training and validation sets. Then, you must plot the loss and accuracy of every magnification, which is crucial for scoring this assignment.

Hint: Training the model for one epoch takes around 7 seconds on the Google Colab environment. You have to train every model for at least 60 epochs, but longer training time can result in higher accuracies.

Note: You are not expected to achieve the exact accuracies reported in the paper for getting the full score. A correct implementation and sufficient training epochs receive the total score for this assignment.

**Configs**

In [95]:
# Implement the training method here (20 points)
max_steps = 200

def train_one_epoch(loader):
    model.train()
    running_loss = 0.0
    running_error = 0
    correct_label_pred = 0
    for batch_num, (data, label) in enumerate(loader):
        if batch_num > max_steps:
            break

        bag_label = label[0]
        data, bag_label = data.to(device), bag_label.to(device)
        data, bag_label = Variable(data), Variable(bag_label)
        data = data.squeeze(0)

        optimizer.zero_grad()

        # calculate loss and metrics
        loss, _ = model.calculate_objective(data, bag_label)
        running_loss += loss.data[0]
        error, predicted_label_train = model.calculate_classification_error(data, bag_label)
        running_error += error

        correct_label_pred += (int(bag_label) == int(predicted_label_train))

        loss.backward()
        optimizer.step()


    running_loss /= max_steps
    running_error /= max_steps
    acc = correct_label_pred / max_steps #(1 - running_error)

    return running_loss, running_error, acc

def validate_one_epoch(loader):
    model.eval()

    running_loss = 0.0
    running_error = 0
    correct_label_pred = 0
    for batch_num, (data, label) in enumerate(loader):

        bag_label = label[0]
        data, bag_label = data.to(device), bag_label.to(device)
        data, bag_label = Variable(data), Variable(bag_label)
        data = data.squeeze(0)

        # calculate loss and metrics
        loss, attention_weights = model.calculate_objective(data, bag_label)
        running_loss += loss.data[0]
        error, predicted_label = model.calculate_classification_error(data, bag_label)
        running_error += error

        correct_label_pred += (int(bag_label) == int(predicted_label))

    running_loss /= len(loader)
    running_error /= len(loader)
    acc = correct_label_pred / len(loader) #(1 - running_error)

    return running_loss, running_error, acc


def train(dataloader):
    # To-Do
    best_val_loss = float('inf')

    for epoch in range(epochs):
      train_loss, train_error, train_acc = train_one_epoch(dataloader['train'])
      val_loss, val_error, val_acc = validate_one_epoch(dataloader['valid'])

      wandb.log({"train_loss": train_loss, "train_acc": train_acc, "val_loss": val_loss, "val_acc": val_acc})

      if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model, model_path)
        torch.save(model.state_dict(), model_path[:-3]+'.pth')


      print(f'Epoch [{epoch+1}/{epochs}] - '
              f'Train Loss: {train_loss.item():.4f} - '
              f'Train Accuracy: {train_acc:.4f} - '
              f'Validation Loss: {val_loss.item():.4f} - '
              f'Validation Accuracy {val_acc:.4f}%')


In [91]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Attention()
model.to(device)

lr = 10e-5
optimizer = optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999), weight_decay=10e-5)

In [92]:
# Train the model on the 40X data here (1 points)
model_path = '/content/best_model_after40X.pt'
epochs = 60
magn = "40X"

wandb.init(
    # set the wandb project where this run will be logged
    project="MIL",

    # track hyperparameters and run metadata
    config={
    "learning_rate": lr,
    "architecture": "Attention-based Deep Multiple Instance Learning",
    "epochs": epochs,
    },
    name=magn
)

train(dataloader[magn])

train_acc,▁▃██
train_loss,█▅▁▁
val_acc,▁▁▁▁
val_loss,▁▁▁▁
train_acc,0.865
train_loss,0.38001
val_acc,0.86869
val_loss,0.35247


Epoch [1/60] - Train Loss: 0.6590 - Train Accuracy: 0.6450 - Validation Loss: 0.6001 - Validation Accuracy 0.7172%
Epoch [2/60] - Train Loss: 0.6035 - Train Accuracy: 0.7100 - Validation Loss: 0.6254 - Validation Accuracy 0.7172%
Epoch [3/60] - Train Loss: 0.6759 - Train Accuracy: 0.6350 - Validation Loss: 0.6297 - Validation Accuracy 0.7273%
Epoch [4/60] - Train Loss: 0.6038 - Train Accuracy: 0.7100 - Validation Loss: 0.5933 - Validation Accuracy 0.7172%
Epoch [5/60] - Train Loss: 0.6084 - Train Accuracy: 0.7050 - Validation Loss: 0.5582 - Validation Accuracy 0.7172%
Epoch [6/60] - Train Loss: 0.5646 - Train Accuracy: 0.7300 - Validation Loss: 0.5922 - Validation Accuracy 0.7172%
Epoch [7/60] - Train Loss: 0.6066 - Train Accuracy: 0.7150 - Validation Loss: 0.5421 - Validation Accuracy 0.7172%
Epoch [8/60] - Train Loss: 0.6152 - Train Accuracy: 0.6850 - Validation Loss: 0.5540 - Validation Accuracy 0.7172%
Epoch [9/60] - Train Loss: 0.5606 - Train Accuracy: 0.6950 - Validation Loss: 0.

In [97]:
# Plot loss and accuracy per epoch for 40X here (2 points)
# loss plot
IPython.display.HTML('<iframe src="https://api.wandb.ai/links/amirezzati/bjcqs568" style="border:none;height:512px;width:1024px">')

In [98]:
# acc plot
IPython.display.HTML('<iframe src="https://api.wandb.ai/links/amirezzati/vk3z07p5" style="border:none;height:512px;width:1024px">')

In [94]:
# Train the model on the 100X data here (1 points)
model_path = '/content/best_model_after100X.pt'
epochs = 60
magn = "100X"

wandb.init(
    # set the wandb project where this run will be logged
    project="MIL",

    # track hyperparameters and run metadata
    config={
    "learning_rate": lr,
    "architecture": "Attention-based Deep Multiple Instance Learning",
    "epochs": epochs,
    },
    name=magn
)
# model = torch.load(model_path)
# model.load_state_dict(torch.load(model_path))
train(dataloader[magn])

train_acc,▁▃▃▃▄▂▅▅▅▄▇▆▅▆█▄▇▇▆▇▅▇▆▇▆▅▇▆▆▆█▅▇▇▇▇▅▆▆█
train_loss,█▆▆▇▆▇▄▄▆▅▂▄▄▃▁▅▄▂▄▂▅▃▄▃▄▄▂▃▃▄▁▄▂▃▂▃▄▃▃▂
val_acc,▁▁▁▁▁▁▇▅▅▆▆▅█▄▆▇▄▄▆█▄█▆▇▇▇▅▅▇▇▂█▇█▆██▆▇▇
val_loss,▇█▇▆▆▆▃▄▃▃▂▃▃▃▂▃▃▃▂▂▃▂▂▂▃▃▂▃▂▂▆▂▂▁▂▁▂▂▂▁
train_acc,0.84
train_loss,0.43967
val_acc,0.84848
val_loss,0.38285


Epoch [1/60] - Train Loss: 0.4744 - Train Accuracy: 0.8050 - Validation Loss: 0.3121 - Validation Accuracy 0.9038%
Epoch [2/60] - Train Loss: 0.4542 - Train Accuracy: 0.8100 - Validation Loss: 0.3286 - Validation Accuracy 0.8750%
Epoch [3/60] - Train Loss: 0.4245 - Train Accuracy: 0.8250 - Validation Loss: 0.3751 - Validation Accuracy 0.8365%
Epoch [4/60] - Train Loss: 0.4458 - Train Accuracy: 0.8200 - Validation Loss: 0.3233 - Validation Accuracy 0.8750%
Epoch [5/60] - Train Loss: 0.4356 - Train Accuracy: 0.8450 - Validation Loss: 0.2737 - Validation Accuracy 0.9038%
Epoch [6/60] - Train Loss: 0.4514 - Train Accuracy: 0.8250 - Validation Loss: 0.3022 - Validation Accuracy 0.8846%
Epoch [7/60] - Train Loss: 0.4438 - Train Accuracy: 0.8300 - Validation Loss: 0.2847 - Validation Accuracy 0.9135%
Epoch [8/60] - Train Loss: 0.4129 - Train Accuracy: 0.8500 - Validation Loss: 0.2942 - Validation Accuracy 0.8750%
Epoch [9/60] - Train Loss: 0.4911 - Train Accuracy: 0.8150 - Validation Loss: 0.

In [100]:
# Plot loss and accuracy per epoch for 100X here (2 points)
# loss plot
IPython.display.HTML('<iframe src="https://api.wandb.ai/links/amirezzati/z1qrxvlc" style="border:none;height:512px;width:1024px">')

In [99]:
# acc plot
IPython.display.HTML('<iframe src="https://api.wandb.ai/links/amirezzati/3gjnbitl" style="border:none;height:512px;width:1024px">')

**RESULT: Best accuracy on validation set is 91.35%**

In [96]:
# Train the model on the 200X data here (1 points)
model_path = '/content/best_model_after200X.pt'
epochs = 60
magn = "200X"

wandb.init(
    # set the wandb project where this run will be logged
    project="MIL",

    # track hyperparameters and run metadata
    config={
    "learning_rate": lr,
    "architecture": "Attention-based Deep Multiple Instance Learning",
    "epochs": epochs,
    },
    name=magn
)

# model = torch.load(model_path)
# model.load_state_dict(torch.load(model_path))
train(dataloader[magn])

train_acc,▃▃▄▅▄▅▇▇▄▄▇▁▄▆▇▂▅▄▄▇▅▄▇▄▇▅▄▆▄▆▆▇▅▄▇▂▅▆█▆
train_loss,▇▇▆▆▆▅▄▃▆▇▄█▆▃▂█▅▆▆▂▅▆▅▇▃▆▆▅▆▄▄▄▅▆▂▇▅▄▁▂
val_acc,█▇▇██▇▇█▇██▇▆▇█▇▇▆█▇▇▇▅▁██▇█▇▇▇▇█▇▇▇▇▇▂▇
val_loss,▂▂▂▁▂▂▁▂▂▂▁▃▃▁▂▂▁▄▂▂▂▁▄█▂▂▂▂▂▂▂▂▂▂▂▂▂▂▇▁
train_acc,0.865
train_loss,0.32743
val_acc,0.89423
val_loss,0.24808


Epoch [1/60] - Train Loss: 0.3581 - Train Accuracy: 0.8700 - Validation Loss: 0.5024 - Validation Accuracy 0.8020%
Epoch [2/60] - Train Loss: 0.2871 - Train Accuracy: 0.9200 - Validation Loss: 0.4398 - Validation Accuracy 0.8317%
Epoch [3/60] - Train Loss: 0.3717 - Train Accuracy: 0.8700 - Validation Loss: 0.4175 - Validation Accuracy 0.8119%
Epoch [4/60] - Train Loss: 0.2678 - Train Accuracy: 0.9150 - Validation Loss: 0.5173 - Validation Accuracy 0.8020%
Epoch [5/60] - Train Loss: 0.3966 - Train Accuracy: 0.8250 - Validation Loss: 0.4267 - Validation Accuracy 0.8218%
Epoch [6/60] - Train Loss: 0.3066 - Train Accuracy: 0.8900 - Validation Loss: 0.4456 - Validation Accuracy 0.8317%
Epoch [7/60] - Train Loss: 0.3317 - Train Accuracy: 0.8550 - Validation Loss: 0.5030 - Validation Accuracy 0.8020%
Epoch [8/60] - Train Loss: 0.3989 - Train Accuracy: 0.8600 - Validation Loss: 0.4515 - Validation Accuracy 0.8020%
Epoch [9/60] - Train Loss: 0.3335 - Train Accuracy: 0.8800 - Validation Loss: 0.

In [ ]:
model = Attention()
model.load_state_dict(torch.load(/content/best_model_after200X.pt'))







In [103]:
# Plot loss and accuracy per epoch for 200X here (2 points)
# loss plot
IPython.display.HTML('<iframe src="https://api.wandb.ai/links/amirezzati/tqqvojth" style="border:none;height:512px;width:1024px">')

In [102]:
# acc plot
IPython.display.HTML('<iframe src="https://api.wandb.ai/links/amirezzati/cn5owk44" style="border:none;height:512px;width:1024px">')

**RESULT: Best accuracy on validation set is 85.15%**

In [101]:
# Train the model on the 400X data here (1 points)
model_path = '/content/best_model_after400X.pt'
epochs = 60
magn = "400X"

wandb.init(
    # set the wandb project where this run will be logged
    project="MIL",

    # track hyperparameters and run metadata
    config={
    "learning_rate": lr,
    "architecture": "Attention-based Deep Multiple Instance Learning",
    "epochs": epochs,
    },
    name=magn
)

# model = torch.load(model_path)
# model.load_state_dict(torch.load(model_path))
train(dataloader[magn])

train_acc,▄▇▇▁▃▃▅▄█▃▅▆▇▅▄▇▆▆▆▅▅▆▄▄▃▅▅▄▅▆▄▆▆▅▆█▅▆▇▆
train_loss,▇▄▄█▆█▆▆▂▇▆▄▃▅▆▄▅▄▃▅▅▄▆▆▆▅▆▅▅▄▆▅▅▅▄▁▅▂▄▃
val_acc,▄▆▄▅▄▄▅▅▄▄▇▇▅▅▁▅▅▇▅▆▅▅▅▅▅█▅▅▅▅▅▅▆▇▅▆▁▅▅▅
val_loss,▆▄▇▃▆▄▂▇█▃▂▃▅▅█▅▃▃▃▁▃▂▃▂▁▃▂▁▁▂▂▃▁▂▂▂▆▅▂▃
train_acc,0.9
train_loss,0.25198
val_acc,0.81188
val_loss,0.41698


Epoch [1/60] - Train Loss: 0.3406 - Train Accuracy: 0.8600 - Validation Loss: 0.3773 - Validation Accuracy 0.8901%
Epoch [2/60] - Train Loss: 0.3168 - Train Accuracy: 0.8650 - Validation Loss: 0.3849 - Validation Accuracy 0.8132%
Epoch [3/60] - Train Loss: 0.2654 - Train Accuracy: 0.9150 - Validation Loss: 0.5600 - Validation Accuracy 0.7363%
Epoch [4/60] - Train Loss: 0.3890 - Train Accuracy: 0.8750 - Validation Loss: 0.3480 - Validation Accuracy 0.8901%
Epoch [5/60] - Train Loss: 0.3112 - Train Accuracy: 0.8850 - Validation Loss: 0.3469 - Validation Accuracy 0.8681%
Epoch [6/60] - Train Loss: 0.3266 - Train Accuracy: 0.8600 - Validation Loss: 0.4308 - Validation Accuracy 0.8132%
Epoch [7/60] - Train Loss: 0.3454 - Train Accuracy: 0.8700 - Validation Loss: 0.3787 - Validation Accuracy 0.8462%
Epoch [8/60] - Train Loss: 0.3733 - Train Accuracy: 0.8350 - Validation Loss: 0.3406 - Validation Accuracy 0.8901%
Epoch [9/60] - Train Loss: 0.3460 - Train Accuracy: 0.8750 - Validation Loss: 0.

In [122]:
# Plot loss and accuracy per epoch for 400X here (2 points)
# loss plot
IPython.display.HTML('<iframe src="https://api.wandb.ai/links/amirezzati/ml6f417g" style="border:none;height:512px;width:1024px">')

In [104]:
# acc plot
IPython.display.HTML('<iframe src="https://api.wandb.ai/links/amirezzati/umew3wh0" style="border:none;height:512px;width:1024px">')

**RESULT: Best accuracy on validation set is 90.11%**

## Evaluation (8 points)

Now, we need to evaluate our best model in every magnification on the test set. Complete the test method and use it for the evaluation of models.

In [107]:
# Implement the test method here (4 points)
def test(dataloader):
    # To-Do
    test_loss, test_error, test_acc = validate_one_epoch(dataloader['test'])

    print(f'Validation Loss: {test_loss.item():.4f} - '
              f'Validation Accuracy {test_acc:.4f}%')


In [112]:
model_path = '/content/best_model_after400X.pt'
model = torch.load(model_path)

In [118]:
# Test the model trained on the 40X data here (1 points)
magn = "40X"
test(dataloader[magn])

Validation Loss: 0.4672 - Validation Accuracy 0.8261%


In [114]:
# Test the model trained on the 100X data here (1 points)
magn = "100X"
test(dataloader[magn])

Validation Loss: 0.2564 - Validation Accuracy 0.9071%


In [115]:
# Test the model trained on the 200X data here (1 points)
magn = "200X"
test(dataloader[magn])

Validation Loss: 0.3119 - Validation Accuracy 0.8671%


In [121]:
# Test the model trained on the 400X data here (1 points)
magn = "400X"
test(dataloader[magn])

Validation Loss: 0.3048 - Validation Accuracy 0.8718%
